In [1]:
import numpy as np
import pandas as pd
import math
from math import sin, cos, radians
import random

import os
import matplotlib.pyplot as plt
import datetime

import scipy.stats as st
import scipy.signal as sgl
pd.set_option('display.max_columns', 500)

#import fastdtw
from scipy.spatial.distance import euclidean
from fastdtw import fastdtw

import scipy.interpolate as spi

#hide warning
import warnings
warnings.filterwarnings('ignore')

In [2]:
"""
The Ramer-Douglas-Peucker algorithm roughly ported from the pseudo-code provided
by http://en.wikipedia.org/wiki/Ramer-Douglas-Peucker_algorithm
"""

from math import sqrt

def distance(a, b):
    return  sqrt((a[0] - b[0]) ** 2 + (a[1] - b[1]) ** 2)

def point_line_distance(point, start, end):
    if (start == end):
        return distance(point, start)
    else:
        n = abs(
            (end[0] - start[0]) * (start[1] - point[1]) - (start[0] - point[0]) * (end[1] - start[1])
        )
        d = sqrt(
            (end[0] - start[0]) ** 2 + (end[1] - start[1]) ** 2
        )
        return n / d

def rdp(points, epsilon):
    """
    Reduces a series of points to a simplified version that loses detail, but
    maintains the general shape of the series.
    """
    dmax = 0.0
    index = 0
    for i in range(1, len(points) - 1):
        d = point_line_distance(points[i], points[0], points[-1])
        if d > dmax:
            index = i
            dmax = d
    if dmax >= epsilon:
        results = rdp(points[:index+1], epsilon)[:-1] + rdp(points[index:], epsilon)
    else:
        results = [points[0], points[-1]]
    return results


def rdp_precoss(df):
    l = []
    for i in range(0, df.shape[0]):
        l.append((df.loc[i, 'X'], df.loc[i, 'Y'],  df.loc[i, 'TStamp'], df.loc[i, 'Pres'], df.loc[i, 'EndPts'],  df.loc[i, 'TStamp2'] ))

    final = rdp(l, 0.0000001)
#     print (f)
    df = pd.DataFrame(final, columns=['X', 'Y', 'TStamp', 'Pres', 'EndPts', 'TStamp2'])
    return df

In [3]:
# remove_duplicated_point
def remove_duplicated_point(df):
    df_new = df.drop(index=df.index)    
    old_x = df['X'][0]
    old_y = df['Y'][0]
    for index, row in df.iterrows():
        if row['X'] != old_x or row['Y']!=old_y or index == 0:
            df_new.loc[len(df_new)] = {'X': row['X'], 'Y': row['Y'], 'TStamp': row['TStamp'], 'Pres': row['Pres'], 'EndPts': row['EndPts'],
                                       'TStamp2': row['TStamp2']}
            old_x = row['X']
            old_y = row['Y']
        # update the EndPts if the point is   
        elif row['X'] == old_x and row['Y'] == old_y and row['EndPts'] == 1:
            df_new.iloc[len(df_new)-1]['EndPts']  = 1
    return df_new

In [4]:
# Cubic-Spline to add points in stroke and make the curves smooth
def interpolate_points(df):
    plt.rcParams['font.sans-serif']=['SimHei'] #用来正常显示中文标签
    plt.rcParams['axes.unicode_minus']=False #用来正常显示负号

    # 按手指离开屏幕的点，分笔触差值，因为在笔触间差值没有意义
    # 寻找断点
    EndPts = [0]
    EndPts.extend(list(df.loc[df["EndPts"] == 1].index))
    EndPts_len = len(EndPts)
#     print(EndPts)
    if EndPts_len < 2:
        EndPts.append(len(df)-1) #如果没有就设最后一个点为离开屏幕的点
        EndPts_len += 1
    
    i = 0
    new_x_all = []
    iy3_x_all = []
    iy3_y_all = []
    print(EndPts)
    while i < EndPts_len - 1:
        #data preparation
        start = EndPts[i]
        end = EndPts[i+1]
        if start == 0:
            X= np.array(df['TStamp2'][0:end].tolist())
            Y= np.array(df['X'][0:end].tolist())
            Y2= np.array(df['Y'][0:end].tolist())
            if df['TStamp2'][start+1:end].max() - df['TStamp2'][start+1:end].min() <=15 or end - start <= 6:
                i+=1
                continue            
            new_x=np.arange(0,df['TStamp2'][0:end].max(),15) #define interpolate points
            
        else:            
            X= np.array(df['TStamp2'][start+1:end].tolist())
            Y= np.array(df['X'][start+1:end].tolist())
            Y2= np.array(df['Y'][start+1:end].tolist())
            if df['TStamp2'][start+1:end].max() - df['TStamp2'][start+1:end].min() <=15 or end - start <= 6:
                i += 1
                continue
            new_x=np.arange(df['TStamp2'][start+1:end].min(),df['TStamp2'][start+1:end].max(),15) #define interpolate points            


        #进行三次样条拟合
        ipo3=spi.splrep(X,Y,k=1, s=10000) #样本点导入，生成参数
        iy3=spi.splev(new_x,ipo3) #根据观测点和样条参数，生成插值
        ipo3_y=spi.splrep(X,Y2,k=1, s=10000) #样本点导入，生成参数
        iy3_y=spi.splev(new_x,ipo3_y) #根据观测点和样条参数，生成插值        
        new_x_all.extend(new_x)
        iy3_x_all.extend(iy3)
        iy3_y_all.extend(iy3_y)
        i += 1

    X_all= np.array(df['TStamp2'].tolist())
    Y_all = np.array(df['X'].tolist())
    Y2_all = np.array(df['Y'].tolist())
       
    df_new = df.copy()
    for i, val in enumerate(new_x_all):
        if val in df['TStamp2'].tolist():
            continue
        if val in df['TStamp2'].tolist():
            continue
        if  abs(iy3_x_all[i]) > 100000 or abs(iy3_y_all[i]) > 100000:
            continue
        df_new.loc[len(df_new)] = {'X': iy3_x_all[i], 'Y': iy3_y_all[i], 'TStamp': 0, 'Pres': 0, 'EndPts': 0,
                                       'TStamp2': val}
    df_new = df_new.sort_values(by=['TStamp2'])
    return df_new

In [5]:
# 求两直线夹角
def get_angle_betw_lines(x1, y1, x2, y2, x3, y3, x4, y4):
    k1 = (y2-y1)/(float(x2-x1))
    k2 = (y4-y3)/(float(x4-x3))
    Cobb = float(math.fabs(np.arctan((k1-k2)/(float(1 + k1*k2)))*180/np.pi)+0.5)
    return Cobb

def get_grivity_angle(P):
    points_left = []
    points_right = []
#     print(list(P.exterior.coords))
    for point in P.exterior.coords:
        if point[0] <= P.centroid.x:
            points_left.append([point[0], point[1]])
        else:
            points_right.append([point[0], point[1]])

    if len(points_left) < 4:
        print(list(P.exterior.coords))
        print(P.centroid)
        
    P_left = Polygon(points_left)
    P_right = Polygon(points_right)        
#     plt.plot(P_left.centroid.y, P_left.centroid.x, 'bx')
#     plt.plot(P_right.centroid.y, P_right.centroid.x, 'bx')

    theta = get_angle_betw_lines( P_left.centroid.x,  P_left.centroid.y,  P_right.centroid.x,  P_right.centroid.y,      
                                 0.0,0.0, 10.0,0.0)
    return theta


#rotate for Polygon
def rotate_polygon(polygon, angle, center_point=(0, 0)):
    """Rotates the given polygon which consists of corners represented as (x,y)
    around center_point (origin by default)
    Rotation is counter-clockwise
    Angle is in degrees
    """
    rotated_polygon = []
    for corner in polygon.exterior.coords:
        rotated_corner = rotate_point(corner, angle, center_point)
        rotated_polygon.append(rotated_corner)
    rotated_polygon = Polygon(rotated_polygon)    
    return rotated_polygon

def rotate_point(point, angle, center_point=(0, 0)):
    """Rotates a point around center_point(origin by default)
    Angle is in degrees.
    Rotation is counter-clockwise
    """
    angle_rad = radians(angle % 360)
    # Shift the point so that center_point becomes the origin
    new_point = (point[0] - center_point[0], point[1] - center_point[1])
    new_point = (new_point[0] * cos(angle_rad) - new_point[1] * sin(angle_rad),
                 new_point[0] * sin(angle_rad) + new_point[1] * cos(angle_rad))
    # Reverse the shifting we have done
    new_point = (new_point[0] + center_point[0], new_point[1] + center_point[1])
    return new_point



# gravity_x, gravity_y = gravity_normalize(file)
# print(gravity_x, gravity_y)

from shapely.geometry import Polygon

def rotate_graphic(file):
    points_left = []
    points_right = []
    points = []

    for index, row in file.iterrows():
        points.append([row['X'], row['Y']])

    P = Polygon(points)
#     print(P.centroid)

    theta = get_grivity_angle(P)
    theta_old = theta
#     print(theta)
    
    limit_rotate_times = 100
    times = 0
    min_theta = theta
    min_theta_times = 0
    if theta >6 and get_grivity_angle(rotate_polygon(P, theta_old/24, (P.centroid.x, P.centroid.y))) < theta: 
        while theta > 6.0 and times <= limit_rotate_times :
    #     if theta <= 90: # clock-wise 顺时针
            P = rotate_polygon(P, theta_old/24, (P.centroid.x, P.centroid.y))        
            theta = get_grivity_angle(P)
            if theta < min_theta:
                min_theta = theta
                min_theta_times = times
            times += 1    
#             print(theta)
        #rotate back to min theta
        if theta >6:
            P = rotate_polygon(P, -theta_old/24*(limit_rotate_times-min_theta_times), (P.centroid.x, P.centroid.y))
            theta = get_grivity_angle(P)
            print(min_theta)
            print(theta)
            
    elif  theta >6 and get_grivity_angle(rotate_polygon(P, -theta_old/24, (P.centroid.x, P.centroid.y))) < theta_old:
        while theta > 6.0 and times <= limit_rotate_times:
            P = rotate_polygon(P, -theta_old/24, (P.centroid.x, P.centroid.y))        
            theta = get_grivity_angle(P)
            if theta < min_theta:
                min_theta = theta
                min_theta_times = times
            times += 1    
#             print(theta)
        #rotate back to min theta
        if theta >6:
            P = rotate_polygon(P, theta_old/24*(limit_rotate_times-min_theta_times), (P.centroid.x, P.centroid.y))
            theta = get_grivity_angle(P)
            print(min_theta)
            print(theta)
            
            
    P_rotated = P
    P_rotated_points_x = []
    P_rotated_points_y = []
    for point in   P_rotated.exterior.coords:
        P_rotated_points_x.append(point[0])   
        P_rotated_points_y.append(point[1])   

#     print(P_rotated.centroid)




    fig = plt.figure(figsize=[6,4])
    plt.scatter(x = file["X"], y = file["Y"], marker='o',c='', edgecolors='g')

#     plt.plot(gravity_y, gravity_x, 'rx')
#     plt.plot(P_left.centroid.y, P_left.centroid.x, 'rx')
#     plt.plot(P_right.centroid.y, P_right.centroid.x, 'rx')

    plt.scatter(x = P_rotated_points_x, y = P_rotated_points_y, marker='o',c='', edgecolors='b')

    points_left = []
    points_right = []
    for point in P_rotated.exterior.coords:
        if point[0] <= P_rotated.centroid.x:
            points_left.append([point[0], point[1]])
        else:
            points_right.append([point[0], point[1]])

    P_left = Polygon(points_left)
    P_right = Polygon(points_right)        
    plt.plot(P_left.centroid.x, P_left.centroid.y, 'rx')
    plt.plot(P_right.centroid.x, P_right.centroid.y, 'rx')

    theta = get_angle_betw_lines( P_left.centroid.x,  P_left.centroid.y,  P_right.centroid.x,  P_right.centroid.y,      
                                 0,0, 10,0)
#     print(theta)
    fig.show()

    file['X'] = P_rotated_points_x[0:-1]
    file['Y'] = P_rotated_points_y[0:-1]
    return file

In [6]:
# Size normalization 大小规整
# position normalization 位置规整
def normalizeFile(normalInputFile):
    widthX = 200 #width
    heightY = 300 #height
    minX = normalInputFile['X'].min()
    minY = normalInputFile['Y'].min()
    maxX = normalInputFile['X'].max()
    maxY = normalInputFile['Y'].max()
    normalInputFile['normalX'] = widthX * ((normalInputFile['X'] - minX)/(maxX - minX))
    normalInputFile['normalY'] = heightY * ((normalInputFile['Y'] - minY)/(maxY - minY))
    
    #position normalization
    averX = normalInputFile['normalX'].mean()
    averY = normalInputFile['normalY'].mean()

    normalInputFile['normalX'] = normalInputFile['normalX'] - averX
    normalInputFile['normalY'] = normalInputFile['normalY'] - averY
    
#     normalInputFile['Angle'] = 0.0
#     normalInputFile['Vel'] = 0.0
#     normalInputFile['Logcr'] = 0.0
#     normalInputFile['Tam'] = 0.0
    return normalInputFile


#长度归一化
def samelen(data,length=400):
#归一化标准长度为 400，若某一签名长度为 d，则需在此签名中每隔
#(d-1)/400 个坐标点提取一个数据， 所取得的数据根据前后两个点进行线性提取    
    data_size = len(data)
    interval = (data_size-1)/length
    start = 0
    new_data = data.drop(index=data.index)
    for dist in np.arange(start, data_size-1, interval):
        first = math.floor(dist)
        second = math.ceil(dist)
        if second >= data_size:
            second = data_size -1
        percent = (dist - first) / 1.0   
        #'X','Y','TStamp','Pres','EndPts'
#         print ((data.X[first] + data.X[second])/2)

#         X = (data.at[first,'X'] + data.at[second,'X']) / 2
#         Y = (data.Y[first] + data.Y[second])/2 
        TStamp = (data.TStamp[first] + data.TStamp[second])/2 
#         normalX = (data.normalX[first] + data.normalX[second])/2
#         normalY = (data.normalY[first] + data.normalY[second])/2 
#         TStamp2 = (data.TStamp2[first] + data.TStamp2[second])/2
        
        X = data.at[first,'X'] + (data.at[second,'X'] - data.at[first,'X']) * percent
        Y = data.at[first,'Y'] + (data.at[second,'Y'] - data.at[first,'Y']) * percent
        normalX = data.at[first,'normalX'] + (data.at[second,'normalX'] - data.at[first,'normalX']) * percent
        normalY = data.at[first,'normalY'] + (data.at[second,'normalY'] - data.at[first,'normalY']) * percent 
        TStamp2 = data.at[first,'TStamp2'] + (data.at[second,'TStamp2'] - data.at[first,'TStamp2']) * percent 
        Pres = data.at[first,'Pres'] + (data.at[second,'Pres'] - data.at[first,'Pres']) * percent 
        Angle = data.at[first,'Angle'] + (data.at[second,'Angle'] - data.at[first,'Angle']) * percent
        Vel = data.at[first,'Vel'] + (data.at[second,'Vel'] - data.at[first,'Vel']) * percent 
        Logcr = data.at[first,'Logcr'] + (data.at[second,'Logcr'] - data.at[first,'Logcr']) * percent 
        Tam = data.at[first,'Tam'] + (data.at[second,'Tam'] - data.at[first,'Tam']) * percent 
        
        if data.EndPts[first] == 1 or data.EndPts[second] == 1:
            EndPts = 1
        else:
            EndPts = 0
            
        new_data.loc[len(new_data)] = {'X': X, 'Y': Y, 'TStamp': TStamp, 'Pres': Pres, 'EndPts': EndPts,'normalX': normalX,
                         'normalY': normalY, 'TStamp2': TStamp2,'Angle': Angle,'Vel': Vel,'Logcr': Logcr,'Tam': Tam }
    return new_data

In [7]:
"""
    add new features to signals
"""

def addSigFeature(df):
    dX = derivation(df['X'].tolist())     # dx
    dY = derivation(df['Y'].tolist())     # dy
#     df['dX'] = dX
#     df['dY'] = dY
    Vel = np.zeros(len(df)).tolist()   # velocity
    Angle = np.zeros(len(df)) # angle
    
    
    T = len(df) -1
    t = 1
    while t<= T:
        Vel[t] = math.sqrt(dX[t] * dX[t] + dY[t] * dY[t])
        if dY[t] != 0 and dX[t] != 0:
            Angle[t] = math.atan(dY[t] / dX[t])
        elif dX[t] == 0:
            Angle[t] = math.atan(dY[t] / 0.01)
        else:
            Angle[t] = 0
        t += 1    
    dAngle = derivation(Angle)
    dVel = derivation(Vel)
    Logcr = np.zeros(len(df)).tolist() 
    Tam = np.zeros(len(df)).tolist() #加速度
    
    t = 1
    while t<= T:
        Logcr[t] = math.log((abs(Vel[t]) + 0.01) / ((abs(dAngle[t]) + 0.01))) ;
        Tam[t] = math.sqrt(dVel[t] * dVel[t] + Vel[t] * Vel[t] * dAngle[t] * dAngle[t]) ;
        t += 1
    df['Angle'] = Angle
    df['Vel'] = Vel
    df['Logcr'] = Logcr
    df['Tam'] = Tam
    return df
    
    
# calculate the difference of the discrete sequence    
def derivation(signal):
    T = len(signal)-1
    dsignal = np.zeros(len(signal)).tolist()
    dsignal[0] = (2*signal[2] + signal[1] - 3*signal[0]) / 5
    dsignal[1] = (2*signal[3] + signal[2] - 2*signal[1] - signal[0]) / 6
    t= 2
    while t<=T-2:
        dsignal[t] = (2*signal[t+2] + signal[t+1] - signal[t-1] - 2*signal[t-2]) / 10 
#         print(dsignal[t])        
        t += 1
        # (2(Xn+2)+(Xn+1)-(Xn-1)-2(Xn-2))/10 == [2(Xn+2)-2(Xn+1)+3(Xn+1)-3(Xn)+3(Xn)-3(Xn-1)+2(Xn-1)-2(Xn-2)]/10
        
    dsignal[T-1] = (signal[T] - signal[T-2] + 2*signal[T-1] - 2*signal[T-3]) / 6 
    dsignal[T] = (3*signal[T] - signal[T-1] - 2*signal[T-2]) / 5 
    return dsignal
    

In [8]:
"""
    calculate the euclidean distance using DTW of two sequences 
"""
def calDistance():
    #set ts_a
    normalX = np.array(data1['normalX'])
    normalY = np.array(data1['normalY'])
    list(zip(normalX,normalY))
    ts_a = np.array(list(zip(normalX,normalY)),dtype=float)
#     print(normalX)

    #set ts_b
    normalX = np.array(data2['normalX'])
    normalY = np.array(data2['normalY'])
    list(zip(normalX,normalY))
    ts_b = np.array(list(zip(normalX,normalY)),dtype=float)

    #set ts_c
    normalX = np.array(data3['normalX'])
    normalY = np.array(data3['normalY'])
    list(zip(normalX,normalY))
    ts_c = np.array(list(zip(normalX,normalY)),dtype=float)


    #the plot of DTW distance for genuine signature
    distance, path = fastdtw(ts_a, ts_b, dist=euclidean)
    print("genuine signature distance: ", distance)    

In [9]:
#preprocessing
def preprocess(df, length=400):
    startTime = df['TStamp'][0]
    df['TStamp2'] = (df['TStamp'] - startTime) #ms
#     df['EndPts'][len(df)-1] = 1
#     print(df)
#     df = rdp_precoss(df) # use RDP algorithm to remove duplicated points
#     df = interpolate_points(df)
    df = remove_duplicated_point(df)    
    df = addSigFeature(df)
    df = normalizeFile(df)
    #     df = rotate_graphic(df)    
    
    df = samelen(df, length)
    df = normalization_df(df)
    return df

def get_ts(df):
    normalX = np.array(df['normalX'])
    normalY = np.array(df['normalY'])
    Pres = np.array(df['Pres'])
    Angle = np.array(df['Angle'])
    Vel = np.array(df['Vel'])
    Logcr = np.array(df['Logcr'])
    Tam = np.array(df['Tam'])
    ts = np.array(list(zip(normalX,normalY,Angle,Vel,Logcr,Tam)),dtype=float)    
#     ts = np.array(list(zip(normalX,normalY,Tam)),dtype=float)    
#     ts = np.array(list(zip(normalX,normalY,Pres)),dtype=float)
#     ts = np.array(list(zip(normalX,normalY,Angle)),dtype=float)    
    return ts

def cal_dfs_max_length(path, files):
    max_length = 0
    for fname in files:
        with open(path + '/' + fname,'rb') as f:
            count = 0
            last_data = '\n'
            while True:
                data = f.read(0x400000)
                if not data:
                    break
                count += data.count(b'\n')
                last_data = data
            if last_data[-1:] != b'\n':
                count += 1 # Remove this if a wc-like count is needed        
        length = count
        if length > max_length:
            max_length = length
    print(max_length-2)        
    return max_length-2  

def normalization_df(df):
    TStamp2 = df['TStamp2'].tolist()
    TStamp = df['TStamp'].tolist()
    normalized_df=(df-df.mean())/df.std()
#     normalized_df=(df-df.min())/(df.max()-df.min())
    normalized_df['TStamp2'] = TStamp2
    normalized_df['TStamp'] = TStamp
    return normalized_df


In [10]:
# read dataset

base_path_forg = 'dataset/SUSIG/FORGERY' #FROM FILE PATH
base_path_genu = 'dataset/SUSIG/GENUINE/SESSION2' #FROM FILE PATH


DEBUG_MODE = True
TOTAL_NUM = 0.0
TOTAL_GENU = 0.0
TOTAL_FORG = 0.0
TOTAL_FN = 0.0
TOTAL_FP = 0.0
TOTAL_ACCU = 0.0
counter = 1

# while (counter<116):
while (counter<116):    
    if (counter<10):
        base1 = "00"
    elif (counter <100):
        base1 = "0"
    else:
        base1 = ""
        
    genuinefiles = []
    forgeryfiles = []
        
    #get genuine signature    
    for root, dirs, files in os.walk(base_path_genu):
        for f in files:
            if f.startswith(base1 + str(counter)):
                genuinefiles = np.append(genuinefiles,f)
                genuinefiles = list(genuinefiles)
    print(genuinefiles)
    #get forgery signature    
    for root, dirs, files in os.walk(base_path_forg):
        for f in files:
            if f.startswith(base1 + str(counter)):
                forgeryfiles = np.append(forgeryfiles,f)
                forgeryfiles = list(forgeryfiles)
    if len(genuinefiles) == 0:
        counter += 1
        continue
        
    i = 0
    genu_train_dfs = [] # genuine data
    genu_train_dists = [] # distances between genuine signature
    genu_train_times = [] # times of genuine signature
    genu_train_files = []
    dfs_max_length = 0 # max length(number of points) in trainning dataset

    
    dist_flag = 0 # 0表示需要重新算
    cal_count = 0
    min_coef_genu_train_dfs = [] # min coef genuine data
    min_coef_genu_train_dists = [] # min coef distances between genuine signature
    min_coef_genu_train_times = [] # min coef times of genuine signature
    min_coef_genu_train_files = []
    min_coef = 1.0 # min coef
    
    while  len(genu_train_files)==0 or dist_flag == 0:
#     genu_train_files = random.sample(genuinefiles, 3)
#     genu_train_files = list(genuinefiles)[0:3]
#     genu_train_files = ['004_1_10.sig', '004_1_2.sig', '004_1_3.sig']
    
#     print(genu_train_files)
        genu_train_files = random.sample(genuinefiles, 3)
        genu_train_dfs = [] # genuine data
        genu_train_dists = [] # distances between genuine signature
        genu_train_times = [] # times of genuine signature
        
        dfs_max_length = cal_dfs_max_length(base_path_genu, genu_train_files)
        for f in genu_train_files:
            temp = str(base_path_genu+"/"+f)
            df = pd.read_csv(temp,delimiter=' ', names=['X','Y','TStamp','Pres','EndPts'], header=None, skiprows=2)        
            df = preprocess(df,dfs_max_length)
            genu_train_times.append(df['TStamp2'][len(df)-1])
            genu_train_dfs.append(df)


        #calculate the threshold distance from training set
        for i in range(0, len(genu_train_dfs)-1):
            data1 = genu_train_dfs[i]
            for j in range(i+1, len(genu_train_dfs)):
                data2 = genu_train_dfs[j]
                #set ts_a
                ts_a = get_ts(data1)

                #set ts_b
                ts_b =  get_ts(data2)
                distance, path = fastdtw(ts_a, ts_b, dist=euclidean)
                if DEBUG_MODE:
                    print("genuine signature distance: ", distance)
                genu_train_dists.append(distance)
        
        cal_count += 1
        
        # 判断是否训练集的dtw距离都小于规定值，如果不小于重新算
        dist_flag = 1
        print('离散系数:',np.std(genu_train_dists)/np.mean(genu_train_dists))
        coef_var = np.std(genu_train_dists)/np.mean(genu_train_dists)
        
        # 计算并保存一个最小的coef值，如果超过最大的cal_count值，则直接把这个值所表示的训练集当做真正的训练集
        if coef_var < min_coef:
            min_coef_genu_train_dfs = genu_train_dfs
            min_coef_genu_train_dists = genu_train_dists
            min_coef_genu_train_times = genu_train_times
            min_coef_genu_train_files = genu_train_files
            min_coef = coef_var
            
        
#         for d in genu_train_dists:
#             if d > 10000.0 or coef_var > 0.2:
        if coef_var > 0.13:
            dist_flag = 0
        
        if dist_flag == 0 and cal_count >= 5: # 超过了max_cal_count的话，就直接选最小的coef,不再继续算了            
            genu_train_dfs = min_coef_genu_train_dfs
            genu_train_dists  = min_coef_genu_train_dists
            genu_train_times  = min_coef_genu_train_times
            genu_train_files  = min_coef_genu_train_files
            dist_flag = 1
#                 break
#     counter = counter + 1
#     continue
    
    
    
                       
    min_train_dist = min(genu_train_dists)
    mean_train_dist = np.mean(genu_train_dists)
#     threshold = min_train_dist * 1.2
    print(np.std(genu_train_dists))
    
    buffer = 0
    if np.std(genu_train_dists) * 2 < 0.2 * mean_train_dist:
        buffer = 0.2 * mean_train_dist
    else:    
        buffer = np.std(genu_train_dists) * 2
        
        
        
    threshold = mean_train_dist + buffer  
    threshold_time = max(genu_train_times)*2 # maximum time of genuine siganature in training set
    
    
#     genuinefiles.extend(forgeryfiles)
#     testfiles = genuinefiles # 先测试假的是否能够通过吧，从简单的开始
    test_dfs = []
    total_num = len(genuinefiles) + len(forgeryfiles) 
    TP = 0.0
    FP = 0.0 
    FN = 0.0
    TN = 0.0
    
    for f in genuinefiles:
        if f in genu_train_files: #去掉训练集的数据
            total_num -= 1
            continue
        print(f)    
        temp = str(base_path_genu+"/"+f)
        df = pd.read_csv(temp,delimiter=' ', names=['X','Y','TStamp','Pres','EndPts'], header=None, skiprows=2) # 去掉超时的
        if (df['TStamp'].max() - df['TStamp'].min()) > threshold_time:
            FN += 1
            continue
        df = preprocess(df,dfs_max_length)
        test_dfs.append(df) 
     
    test_dfs_geni_num = len(test_dfs)
    for f in forgeryfiles:
        temp = str(base_path_forg+"/"+f)
        print(f) 
        df = pd.read_csv(temp,delimiter=' ', names=['X','Y','TStamp','Pres','EndPts'], header=None, skiprows=2)
        if (df['TStamp'].max() - df['TStamp'].min()) > threshold_time:
            TN += 1
            continue
        df = preprocess(df,dfs_max_length)
        test_dfs.append(df)
    
    
    #calculate the min distance from each test to training set, if d <= threshold, we say it is genuine, otherwise forgery 
    for i in range(0, len(test_dfs)):
        data1 = test_dfs[i]
        test_dists = []
        for j in range(0, len(genu_train_dfs)):
            data2 = genu_train_dfs[j]
            #set ts_a
            ts_a =  get_ts(data1)
            #set ts_b
            ts_b =  get_ts(data2)
            distance, path = fastdtw(ts_a, ts_b, dist=euclidean)
            test_dists.append(distance)
            
        min_test_dist = min(test_dists)
        max_test_dist = max(test_dists)
        mean_test_dist = np.mean(test_dists)
        test_dist_chosen = mean_test_dist # choose the mean of test distances
        if min_test_dist == 0:
            total_num -= 1
            print(total_num)
            continue
            
        if DEBUG_MODE:    
            print("test signature min distance: ", min_test_dist," mean distance: ", mean_test_dist, " threshold: ", threshold)
        if test_dist_chosen > threshold  and i<test_dfs_geni_num:
            FN += 1.0
        elif  test_dist_chosen > threshold and i>=test_dfs_geni_num:
            TN += 1.0
        elif  test_dist_chosen <= threshold and i<test_dfs_geni_num:
            TP += 1.0
        elif  test_dist_chosen <= threshold and i>=test_dfs_geni_num:
            FP += 1.0
            
#         if DEBUG_MODE:    
#             print("test signature min distance: ", min_test_dist, "max distance: ", max_test_dist, " threshold: ", threshold)
#         if max_test_dist > threshold  and i<test_dfs_geni_num:
#             FN += 1.0
#         elif  min_test_dist > threshold and i>=test_dfs_geni_num:
#             TN += 1.0
#         elif  max_test_dist <= threshold and i<test_dfs_geni_num:
#             TP += 1.0
#         elif  min_test_dist <= threshold and i>=test_dfs_geni_num:
#             FP += 1.0            
            
            
    genu_test_num = 7
    forg_test_num = 10
    accuracy = (TP + TN)/total_num * 100
    FRR = FN/genu_test_num *100 # 真的没通过
    FAR = FP/forg_test_num *100 #假的通过了
    
    TOTAL_NUM += total_num
    TOTAL_GENU += genu_test_num
    TOTAL_FORG += forg_test_num
    TOTAL_FN += FN
    TOTAL_FP += FP
    TOTAL_FRR = TOTAL_FN/TOTAL_GENU * 100
    TOTAL_FAR = TOTAL_FP/TOTAL_FORG *100
    
    TOTAL_ACCU = (TOTAL_NUM - TOTAL_FN - TOTAL_FP) / TOTAL_NUM * 100
    print("test num: " + str(total_num) + ", accuracy: " + str(accuracy) + "%")
    print("FRR: " + str(FRR) + "%, FAR: " + str(FAR)+ "%")
    print("TOTAL_FRR: " + str(TOTAL_FRR) + "%, TOTAL_FAR: " + str(TOTAL_FAR)+ "%, TOTAL_TEST_SIGN:" + 
          str(TOTAL_NUM/(genu_test_num + forg_test_num)) + "TOTAL_ACCU: " + str(TOTAL_ACCU) + "%")
    print("\n") 
    
    counter = counter + 1

['001_2_1.sig', '001_2_10.sig', '001_2_2.sig', '001_2_3.sig', '001_2_4.sig', '001_2_5.sig', '001_2_6.sig', '001_2_7.sig', '001_2_8.sig', '001_2_9.sig']
181
genuine signature distance:  150.49003299701218
genuine signature distance:  158.76969521646464
genuine signature distance:  163.5059301831239
离散系数: 0.03413289751570033
5.378953922331255
001_2_1.sig
001_2_10.sig
001_2_3.sig
001_2_6.sig
001_2_7.sig
001_2_8.sig
001_2_9.sig
001_f_1.sig
001_f_10.sig
001_f_2.sig
001_f_3.sig
001_f_4.sig
001_f_5.sig
001_f_6.sig
001_f_7.sig
001_f_8.sig
001_f_9.sig
test signature min distance:  144.45856811450625  mean distance:  163.67262041875836  threshold:  189.1062633586403
test signature min distance:  155.14231862573686  mean distance:  170.34025577324525  threshold:  189.1062633586403
test signature min distance:  177.57146378219937  mean distance:  186.01016393879345  threshold:  189.1062633586403
test signature min distance:  147.59352927195252  mean distance:  161.66762799920528  threshold:  189.1

test signature min distance:  174.10769604347797  mean distance:  176.3878609122938  threshold:  130.81418653009945
test signature min distance:  173.20012187086346  mean distance:  180.38305825768546  threshold:  130.81418653009945
test signature min distance:  230.53533833335732  mean distance:  239.92534465043028  threshold:  130.81418653009945
test signature min distance:  194.29913969898607  mean distance:  202.54431412305925  threshold:  130.81418653009945
test signature min distance:  192.33307547169665  mean distance:  205.0241036861346  threshold:  130.81418653009945
test signature min distance:  210.68597178304205  mean distance:  220.49432144440553  threshold:  130.81418653009945
test num: 17, accuracy: 82.35294117647058%
FRR: 42.857142857142854%, FAR: 0.0%
TOTAL_FRR: 10.714285714285714%, TOTAL_FAR: 10.0%, TOTAL_TEST_SIGN:4.0TOTAL_ACCU: 89.70588235294117%


[]
[]
[]
['008_2_1.sig', '008_2_10.sig', '008_2_2.sig', '008_2_3.sig', '008_2_4.sig', '008_2_5.sig', '008_2_6.sig', '00

genuine signature distance:  598.3935384866711
genuine signature distance:  503.57227851073833
离散系数: 0.0715034042486301
39.11390036500856
011_2_2.sig
011_2_4.sig
011_2_5.sig
011_2_6.sig
011_2_7.sig
011_2_8.sig
011_2_9.sig
011_f_1.sig
011_f_10.sig
011_f_2.sig
011_f_3.sig
011_f_4.sig
011_f_5.sig
011_f_6.sig
011_f_7.sig
011_f_8.sig
011_f_9.sig
test signature min distance:  448.0114412904394  mean distance:  488.98935213382737  threshold:  656.4258154031809
test signature min distance:  460.956440774554  mean distance:  513.2017891601107  threshold:  656.4258154031809
test signature min distance:  471.2957997650518  mean distance:  494.2391765889926  threshold:  656.4258154031809
test signature min distance:  468.1125907083827  mean distance:  491.54303451122934  threshold:  656.4258154031809
test signature min distance:  505.1198985792794  mean distance:  541.367270396232  threshold:  656.4258154031809
test signature min distance:  531.3478756395683  mean distance:  540.1936853102553  thr

015_f_10.sig
015_f_2.sig
015_f_3.sig
015_f_4.sig
015_f_5.sig
015_f_6.sig
015_f_7.sig
015_f_8.sig
015_f_9.sig
test signature min distance:  207.13428281982743  mean distance:  217.89136289824123  threshold:  300.019657972441
test signature min distance:  236.2226274339295  mean distance:  252.33827970622292  threshold:  300.019657972441
test signature min distance:  176.3347081363288  mean distance:  214.82558211687675  threshold:  300.019657972441
test signature min distance:  189.16197552045153  mean distance:  206.2951332556482  threshold:  300.019657972441
test signature min distance:  177.5699935565159  mean distance:  200.4967719219062  threshold:  300.019657972441
test signature min distance:  177.28196768683722  mean distance:  206.61126193375821  threshold:  300.019657972441
test signature min distance:  188.241153073879  mean distance:  215.66716076608722  threshold:  300.019657972441
test signature min distance:  318.06777492955575  mean distance:  322.8693537868475  threshol

test signature min distance:  304.2809194826209  mean distance:  313.6346698718963  threshold:  332.5677315508763
test signature min distance:  303.50667729750586  mean distance:  325.68287714916073  threshold:  332.5677315508763
test signature min distance:  483.8109750038223  mean distance:  493.05030888473607  threshold:  332.5677315508763
test signature min distance:  469.2668349229578  mean distance:  489.1183210026136  threshold:  332.5677315508763
test signature min distance:  440.67125098889136  mean distance:  453.12165158072736  threshold:  332.5677315508763
test num: 17, accuracy: 76.47058823529412%
FRR: 0.0%, FAR: 40.0%
TOTAL_FRR: 11.224489795918368%, TOTAL_FAR: 12.857142857142856%, TOTAL_TEST_SIGN:14.0TOTAL_ACCU: 87.81512605042016%


['020_2_1.sig', '020_2_10.sig', '020_2_2.sig', '020_2_3.sig', '020_2_4.sig', '020_2_5.sig', '020_2_6.sig', '020_2_7.sig', '020_2_8.sig', '020_2_9.sig']
376
genuine signature distance:  289.8911346258925
genuine signature distance:  284.6764515

...

095_2_10.sig
095_2_3.sig
095_2_5.sig
095_2_6.sig
095_2_7.sig
095_2_8.sig
095_f_1.sig
095_f_10.sig
095_f_2.sig
095_f_3.sig
095_f_4.sig
095_f_5.sig
095_f_6.sig
095_f_7.sig
095_f_8.sig
095_f_9.sig
test signature min distance:  290.10884266723616  mean distance:  317.2724031837547  threshold:  285.6599024640868
test signature min distance:  211.15496165798658  mean distance:  233.20308396307226  threshold:  285.6599024640868
test signature min distance:  181.6179374551984  mean distance:  213.815855574514  threshold:  285.6599024640868
test signature min distance:  235.14773944805165  mean distance:  269.5662520266367  threshold:  285.6599024640868
test signature min distance:  260.50071846157005  mean distance:  272.29101397053256  threshold:  285.6599024640868
test signature min distance:  275.40442025477876  mean distance:  283.06604071257345  threshold:  285.6599024640868
test signature min distance:  195.7448653365432  mean distance:  220.9264086943143  threshold:  285.6599024640868
t

genuine signature distance:  565.7078168227437
genuine signature distance:  516.6915838147216
genuine signature distance:  496.01242728661856
离散系数: 0.055549086937721814
29.226445284294446
099_2_1.sig
099_2_2.sig
099_2_3.sig
099_2_4.sig
099_2_5.sig
099_2_6.sig
099_2_9.sig
099_f_1.sig
099_f_10.sig
099_f_2.sig
099_f_3.sig
099_f_4.sig
099_f_5.sig
099_f_6.sig
099_f_7.sig
099_f_8.sig
099_f_9.sig
test signature min distance:  472.15606525945435  mean distance:  493.8073898886765  threshold:  631.3647311696335
test signature min distance:  479.75762765502094  mean distance:  505.4020397705815  threshold:  631.3647311696335
test signature min distance:  463.58814852283723  mean distance:  477.63208413372746  threshold:  631.3647311696335
test signature min distance:  482.09726458433164  mean distance:  505.1701381348527  threshold:  631.3647311696335
test signature min distance:  514.0740911988196  mean distance:  525.156432886708  threshold:  631.3647311696335
test signature min distance:  480

test signature min distance:  224.8882840669436  mean distance:  231.2444850876275  threshold:  253.60980983286737
test signature min distance:  166.80465265308916  mean distance:  194.45363135292268  threshold:  253.60980983286737
test signature min distance:  208.0626974743931  mean distance:  214.45265345268035  threshold:  253.60980983286737
test num: 17, accuracy: 100.0%
FRR: 0.0%, FAR: 0.0%
TOTAL_FRR: 10.671256454388985%, TOTAL_FAR: 5.1807228915662655%, TOTAL_TEST_SIGN:83.0TOTAL_ACCU: 92.55846917080085%


['104_2_1.sig', '104_2_10.sig', '104_2_2.sig', '104_2_3.sig', '104_2_4.sig', '104_2_5.sig', '104_2_6.sig', '104_2_7.sig', '104_2_8.sig', '104_2_9.sig']
156
genuine signature distance:  230.7739432847402
genuine signature distance:  224.26454645933902
genuine signature distance:  220.155160724095
离散系数: 0.019424882346090792
4.371852407057278
104_2_1.sig
104_2_10.sig
104_2_2.sig
104_2_3.sig
104_2_5.sig
104_2_7.sig
104_2_8.sig
104_f_1.sig
104_f_10.sig
104_f_2.sig
104_f_3.sig
104_f_4

genuine signature distance:  234.7044229989303
genuine signature distance:  282.055493903097
离散系数: 0.09230608829708176
24.871482892023653
108_2_1.sig
108_2_10.sig
108_2_2.sig
108_2_4.sig
108_2_7.sig
108_2_8.sig
108_2_9.sig
108_f_1.sig
108_f_10.sig
108_f_2.sig
108_f_3.sig
108_f_4.sig
108_f_5.sig
108_f_6.sig
108_f_7.sig
108_f_8.sig
108_f_9.sig
test signature min distance:  235.62958788473435  mean distance:  253.42893453240404  threshold:  323.3348961161856
test signature min distance:  229.26889581048258  mean distance:  256.60789165507236  threshold:  323.3348961161856
test signature min distance:  234.68345877964802  mean distance:  248.1207564698474  threshold:  323.3348961161856
test signature min distance:  229.91462035835943  mean distance:  251.7348302738907  threshold:  323.3348961161856
test signature min distance:  170.32726156111445  mean distance:  224.4211507527725  threshold:  323.3348961161856
test signature min distance:  212.18880954854316  mean distance:  266.081270696

test signature min distance:  101.05665752029928  mean distance:  115.30845412420739  threshold:  154.52370376937796
test signature min distance:  94.83143705124571  mean distance:  121.94438379315166  threshold:  154.52370376937796
test signature min distance:  92.4325359971627  mean distance:  128.9574318380494  threshold:  154.52370376937796
test signature min distance:  189.99229457951176  mean distance:  199.06106667946574  threshold:  154.52370376937796
test signature min distance:  164.3106733239911  mean distance:  176.10622217079208  threshold:  154.52370376937796
test signature min distance:  189.6863161685868  mean distance:  200.47566768171714  threshold:  154.52370376937796
test signature min distance:  186.7629319190093  mean distance:  193.77492778282408  threshold:  154.52370376937796
test signature min distance:  178.3067262915486  mean distance:  197.53350418447016  threshold:  154.52370376937796
test signature min distance:  170.10996261021123  mean distance:  185.12